# **Julia** workshop

![](JuliaLogo.jpeg)

# Level of this workshop: **Beginner**

# __(You are all welcome to work together, learn)__

> ## In this workshop, you will see:

- ### a brief background of Julia Language
- ### what are the advantages implemented in Julia
- ### how the julia community works
- ### some applications in chemistry
- ### some applications in machine learning
- ### a brief and fun hands-on

### The objective of this workshop is **not** to make you an expert in Julia, but to inspire you starting applying julia in your projects, studies or research!

![](JuliaHistory.png)

### Julia language is a compiled programming language released in 2012!

In [ ]:
## **Curious fact** Why the programming language is called Julia

## 1. How to use libraries in Julia

#### 1. Use **Pkg** manager

In [28]:
using Pkg

##### 1.1. **Extra topic**: Let's learn how to use Pkg on REPL (Read-Eval-Print Loop)

In [29]:
Pkg.add("Molly")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


## 2. Our **first simulation**

> Here we have a fluid acting under the Lennard-Jones potential to start with.

In [30]:
using Molly

#### 2.1. Using **100 atoms**

In [31]:
n_atoms = 100 #(this is the number of atoms)

100

In [32]:
atom_mass = 10.0u"u"

10.0 u

In [33]:
atoms = [Atom(mass=atom_mass, σ=0.3u"nm", ϵ=0.2u"kJ * mol^-1") for i in 1:n_atoms]

100-element Vector{Atom{Float64, Quantity{Float64, 𝐌, Unitful.FreeUnits{(u,), 𝐌, nothing}}, Quantity{Float64, 𝐋, Unitful.FreeUnits{(nm,), 𝐋, nothing}}, Quantity{Float64, 𝐋² 𝐌 𝐍⁻¹ 𝐓⁻², Unitful.FreeUnits{(kJ, mol⁻¹), 𝐋² 𝐌 𝐍⁻¹ 𝐓⁻², nothing}}}}:
 Atom with index 1, charge=0.0, mass=10.0 u, σ=0.3 nm, ϵ=0.2 kJ mol⁻¹
 Atom with index 1, charge=0.0, mass=10.0 u, σ=0.3 nm, ϵ=0.2 kJ mol⁻¹
 Atom with index 1, charge=0.0, mass=10.0 u, σ=0.3 nm, ϵ=0.2 kJ mol⁻¹
 Atom with index 1, charge=0.0, mass=10.0 u, σ=0.3 nm, ϵ=0.2 kJ mol⁻¹
 Atom with index 1, charge=0.0, mass=10.0 u, σ=0.3 nm, ϵ=0.2 kJ mol⁻¹
 Atom with index 1, charge=0.0, mass=10.0 u, σ=0.3 nm, ϵ=0.2 kJ mol⁻¹
 Atom with index 1, charge=0.0, mass=10.0 u, σ=0.3 nm, ϵ=0.2 kJ mol⁻¹
 Atom with index 1, charge=0.0, mass=10.0 u, σ=0.3 nm, ϵ=0.2 kJ mol⁻¹
 Atom with index 1, charge=0.0, mass=10.0 u, σ=0.3 nm, ϵ=0.2 kJ mol⁻¹
 Atom with index 1, charge=0.0, mass=10.0 u, σ=0.3 nm, ϵ=0.2 kJ mol⁻¹
 ⋮
 Atom with index 1, charge=0.0, mass=10.0 u, σ=0.3 nm, 

> the units are re-exported using Unitful.jl. Let's take a look at the syntax.

In [34]:
Pkg.add("Unitful")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [35]:
using Unitful

In [36]:
### bla bla bla

In [37]:
Pkg.add("AtomsBase")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [38]:
atoms = [Atom(mass=1.0f0), Atom(mass=1.0f0)]
coords = [SVector(0.3f0, 0.5f0), SVector(0.7f0, 0.5f0)]
velocities = [SVector(0.0f0, 1.0f0), SVector(0.0f0, -1.0f0)]
pairwise_inters = (Gravity(nl_only=false, G=1.5f0),)
simulator = VelocityVerlet(dt=0.002f0)
boundary = RectangularBoundary(1.0f0, 1.0f0)

sys = System(
    atoms=atoms,
    pairwise_inters=pairwise_inters,
    coords=coords,
    velocities=velocities,
    boundary=boundary,
    loggers=(coords=CoordinateLogger(Float32, 10; dims=2),),
    force_units=NoUnits,
    energy_units=NoUnits,
)

simulate!(sys, simulator, 2_000)

System with 2 atoms, boundary RectangularBoundary{Float32}(Float32[1.0, 1.0])

In [39]:
visualize(
    sys.loggers.coords,
    boundary,
    "sim_gravity.mp4";
    trails=4,
    framerate=15,
    color=[:orange, :lightgreen],
)

"sim_gravity.mp4"

#### Variations of the Morse potential

In [40]:
using Molly
using GLMakie

boundary = CubicBoundary(5.0, 5.0, 5.0)
dists = collect(0.12:0.005:2.0)

function energies(inter)
    return map(dists) do dist
        c1 = SVector(1.0, 1.0, 1.0)
        c2 = SVector(dist + 1.0, 1.0, 1.0)
        potential_energy(inter, c1, c2, boundary)
    end
end

f = Figure(resolution=(600, 400))
ax = Axis(
    f[1, 1],
    xlabel="Distance / nm",
    ylabel="Potential energy / kJ * mol^-1",
    title="Variations of the Morse potential",
)
lines!(
    ax,
    dists,
    energies(HarmonicBond(k=20_000.0, r0=0.2)),
    label="Harmonic",
)
for a in [2.5, 5.0, 10.0]
    lines!(
        ax,
        dists,
        energies(MorseBond(D=100.0, a=a, r0=0.2)),
        label="Morse a=$a nm^-1",
    )
end
ylims!(ax, 0.0, 120.0)
axislegend(position=:rb)
save("morse.png", f)

UndefKeywordError: UndefKeywordError: keyword argument b0 not assigned

### A **fun topic**: simulating the solar system

In [41]:
using GLMakie

# Using get_body_barycentric_posvel from Astropy
coords = [
    SVector(-1336052.8665050615,  294465.0896030796 ,  158690.88781384667)u"km",
    SVector(-58249418.70233503 , -26940630.286818042, -8491250.752464907 )u"km",
    SVector( 58624128.321813114, -81162437.2641475  , -40287143.05760552 )u"km",
    SVector(-99397467.7302648  , -105119583.06486066, -45537506.29775053 )u"km",
    SVector( 131714235.34070954, -144249196.60814604, -69730238.5084304  )u"km",
]

velocities = [
    SVector(-303.86327859262457, -1229.6540090943934, -513.791218405548  )u"km * d^-1",
    SVector( 1012486.9596885007, -3134222.279236384 , -1779128.5093088674)u"km * d^-1",
    SVector( 2504563.6403826815,  1567163.5923297722,  546718.234192132  )u"km * d^-1",
    SVector( 1915792.9709661514, -1542400.0057833872, -668579.962254351  )u"km * d^-1",
    SVector( 1690083.43357355  ,  1393597.7855017239,  593655.0037930267 )u"km * d^-1",
]

body_masses = [
    1.989e30u"kg",
    0.330e24u"kg",
    4.87e24u"kg" ,
    5.97e24u"kg" ,
    0.642e24u"kg",
]

boundary = CubicBoundary(1e9u"km", 1e9u"km", 1e9u"km")

# Convert the gravitational constant to the appropriate units
inter = Gravity(G=convert(typeof(1.0u"km^3 * kg^-1 * d^-2"), Unitful.G))

sys = System(
    atoms=[Atom(mass=m) for m in body_masses],
    pairwise_inters=(inter,),
    coords=coords .+ (SVector(5e8, 5e8, 5e8)u"km",),
    velocities=velocities,
    boundary=boundary,
    loggers=(coords=CoordinateLogger(typeof(1.0u"km"), 10),),
    force_units=u"kg * km * d^-2",
    energy_units=u"kg * km^2 * d^-2",
)

simulator = Verlet(
    dt=0.1u"d",
    remove_CM_motion=false,
)

simulate!(sys, simulator, 3650) # 1 year

visualize(
    sys.loggers.coords,
    boundary,
    "sim_planets.mp4";
    trails=5,
    color=[:yellow, :grey, :orange, :blue, :red],
    markersize=[0.25, 0.08, 0.08, 0.08, 0.08],
    transparency=false,
)


"sim_planets.mp4"

In [42]:
using Molly
using GLMakie
using LinearAlgebra

struct BondableAtom
    i::Int
    mass::Float64
    σ::Float64
    ϵ::Float64
    partners::Set{Int}
end

Molly.mass(ba::BondableAtom) = ba.mass

struct BondableInteraction <: PairwiseInteraction
    nl_only::Bool
    prob_formation::Float64
    prob_break::Float64
    dist_formation::Float64
    k::Float64
    r0::Float64
end

function Molly.force(inter::BondableInteraction,
                        dr,
                        coord_i,
                        coord_j,
                        atom_i,
                        atom_j,
                        boundary)
    # Break bonds randomly
    if atom_j.i in atom_i.partners && rand() < inter.prob_break
        delete!(atom_i.partners, atom_j.i)
        delete!(atom_j.partners, atom_i.i)
    end
    # Make bonds between close atoms randomly
    r2 = sum(abs2, dr)
    if r2 < inter.r0 * inter.dist_formation && rand() < inter.prob_formation
        push!(atom_i.partners, atom_j.i)
        push!(atom_j.partners, atom_i.i)
    end
    # Apply the force of a harmonic bond
    if atom_j.i in atom_i.partners
        c = inter.k * (norm(dr) - inter.r0)
        fdr = -c * normalize(dr)
        return fdr
    else
        return zero(coord_i)
    end
end

function bonds(sys::System, neighbors=nothing, n_threads::Integer=Threads.nthreads())
    bonds = BitVector()
    for i in 1:length(sys)
        for j in 1:(i - 1)
            push!(bonds, j in sys.atoms[i].partners)
        end
    end
    return bonds
end

BondLogger(n_steps) = GeneralObservableLogger(bonds, BitVector, n_steps)

n_atoms = 200
boundary = RectangularBoundary(10.0, 10.0)
n_steps = 2_000
temp = 1.0

atoms = [BondableAtom(i, 1.0, 0.1, 0.02, Set([])) for i in 1:n_atoms]
coords = place_atoms(n_atoms, boundary, min_dist=0.1)
velocities = [velocity(1.0, temp; dims=2) for i in 1:n_atoms]
pairwise_inters = (
    SoftSphere(nl_only=true),
    BondableInteraction(true, 0.1, 0.1, 1.1, 2.0, 0.1),
)
neighbor_finder = DistanceNeighborFinder(
    nb_matrix=trues(n_atoms, n_atoms),
    n_steps=10,
    dist_cutoff=2.0,
)
simulator = VelocityVerlet(
    dt=0.02,
    coupling=AndersenThermostat(temp, 5.0),
)

sys = System(
    atoms=atoms,
    pairwise_inters=pairwise_inters,
    coords=coords,
    velocities=velocities,
    boundary=boundary,
    neighbor_finder=neighbor_finder,
    loggers=(
        coords=CoordinateLogger(Float64, 20; dims=2),
        bonds=BondLogger(20),
    ),
    force_units=NoUnits,
    energy_units=NoUnits,
)

simulate!(sys, simulator, n_steps)

connections = Tuple{Int, Int}[]
for i in 1:length(sys)
    for j in 1:(i - 1)
        push!(connections, (i, j))
    end
end

visualize(
    sys.loggers.coords,
    boundary,
    "sim_mutbond.mp4";
    connections=connections,
    connection_frames=values(sys.loggers.bonds),
    markersize=0.1,
)

MethodError: MethodError: no method matching place_atoms(::Int64, ::RectangularBoundary{Float64}; min_dist=0.1)
Closest candidates are:
  place_atoms(::Integer, ::Any, !Matched::Any; max_attempts) at ~/.julia/packages/Molly/RD5GY/src/setup.jl:22 got unsupported keyword argument "min_dist"